# Расширенное тестирование модуля формирования портфеля

Этот ноутбук демонстрирует расширенный цикл работы с модулем tvr_service.pipeline.portfolio — от загрузки данных по инструментам до расчёта распределения капитала с использованием индикаторного инструмента и параметров.


## 1. Подготовка окружения

Импортируем необходимые библиотеки и подключим модуль портфеля. Добавляем каталог src в sys.path, чтобы можно было использовать код проекта без установки пакета.


In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from tvr_service.pipeline import (
    allocations_to_frame,
    build_portfolio,
    filter_by_suffix,
    filter_by_whitelist,
    load_securities,
    load_whitelist,
)

import pandas as pd


## 2. Настройка параметров теста

Задаём базовые параметры: объём капитала, необходимость скачивания свежих данных с MOEX, путь к локальному CSV (если он есть), фильтрацию по суффиксу тикера, количество параметров и индикаторный инструмент.


In [2]:
CAPITAL = 3_000_000  # общий капитал, руб.
PREFER_REMOTE = True   # при True сначала пробуем скачать данные с MOEX
SEC_DATA_FILE = None   # можно указать путь к локальному sec_tvr.csv
SUFFIX_FILTER = 'Z5'   # например, 'H5', 'M5' и т.п. для фьючерсов
NUM_PARAMS = 3         # количество параметров для каждого инструмента в портфеле
SEC_IND = 'MXZ5'       # код индикаторного инструмента (пример)

# Параметры для расчета float значений
float_1_x = 0.033  # коэффициент для float_1
float_2_x = 0.01   # коэффициент для float_2
float_3_x = 0.01   # значение для float_3
float_4_x = 0.05   # значение для float_4


## 3. Загрузка справочника инструментов

Попробуем получить таблицу инструментов. Функция load_securities автоматически рассчитает поле full_price и добавит тикеры TI* (клонов TB*). В процессе будет использован список из интернета или локального файла.


In [3]:
securities = load_securities(sec_data_file=SEC_DATA_FILE, prefer_remote=PREFER_REMOTE)
print(f"Загружено инструментов: {len(securities)}")
securities.head()


Загружено инструментов: 400


,SECID,MINSTEP,STEPPRICE,PREVSETTLEPRICE,INITIALMARGIN,BUYSELLFEE,SCALPERFEE,SHORTNAME,full_price,CODE,PRICE,SELLDEPO,base_code
0,AEH6,0.001,1.0,24.550,3859.08,1.13,0.57,AED-3.26,24550.0,AEH6,24.550,3859.08,AED
1,AEM6,0.001,1.0,25.120,4040.33,1.16,0.58,AED-6.26,25120.0,AEM6,25.120,4040.33,AED
2,AEZ5,0.001,1.0,23.478,3759.46,1.08,0.54,AED-12.25,23478.0,AEZ5,23.478,3759.46,AED
3,AFH6,1.000,1.0,6020.000,1611.87,1.20,0.60,AFLT-3.26,6020.0,AFH6,6020.000,1611.87,AFLT
4,AFZ5,1.000,1.0,5793.000,1523.15,1.15,0.58,AFLT-12.25,5793.0,AFZ5,5793.000,1523.15,AFLT


## 4. Получение цены индикаторного инструмента

Найдем цену индикаторного инструмента в справочнике и сохраним её в переменную SEC_IND_PRICE.


In [4]:
# Поиск индикаторного инструмента в справочнике
indicator_security = securities[securities['SECID'] == SEC_IND]

if not indicator_security.empty:
    SEC_IND_PRICE = indicator_security['full_price'].iloc[0]
    print(f"Индикаторный инструмент {SEC_IND} найден. Цена: {SEC_IND_PRICE:,.2f} руб.")
    print(f"Краткое название: {indicator_security['SHORTNAME'].iloc[0]}")
else:
    print(f"Индикаторный инструмент {SEC_IND} не найден в справочнике!")
    print("Доступные инструменты с похожими кодами:")
    similar_securities = securities[securities['SECID'].str.contains(SEC_IND[:3], case=False)]
    if not similar_securities.empty:
        print(similar_securities[['SECID', 'SHORTNAME']].head())
    else:
        print("Похожих инструментов не найдено.")
    SEC_IND_PRICE = None


Индикаторный инструмент MXZ5 найден. Цена: 281,500.00 руб.
Краткое название: MIX-12.25


## 5. Фильтрация справочника

Сначала ограничиваем список инструментов по выбранному `SUFFIX_FILTER`, затем применяем whitelist. Так можно сузить вселенную до нужной серии и далее оставить только интересующие базовые активы.


In [5]:
try:
    whitelist = load_whitelist()
    print(f'Инструментов в whitelist: {len(whitelist)}')
except FileNotFoundError:
    whitelist = None
    print('Файл whitelist не найден. Будем использовать полную вселенную инструментов.')

# Фильтрация по суффиксу
base_selection = filter_by_suffix(securities, SUFFIX_FILTER)
print(f'Инструментов после фильтра по суффиксу {SUFFIX_FILTER}: {len(base_selection)}')

# Фильтрация по whitelist
if whitelist and not base_selection.empty:
    filtered_securities = filter_by_whitelist(base_selection, whitelist)
    print(f'Инструментов после применения whitelist: {len(filtered_securities)}')
elif whitelist:
    filtered_securities = pd.DataFrame(columns=base_selection.columns if not base_selection.empty else securities.columns)
    print('После фильтра по суффиксу подходящих инструментов нет.')
else:
    filtered_securities = base_selection

if not filtered_securities.empty:
    print('Примеры отфильтрованных инструментов:')
    display(filtered_securities[['SECID', 'SHORTNAME', 'base_code', 'full_price']].head())
    
    # Проверка уникальности по base_code
    print(f"\nПроверка уникальности по base_code:")
    unique_base_codes = filtered_securities['base_code'].nunique()
    total_instruments = len(filtered_securities)
    print(f"Уникальных base_code: {unique_base_codes}")
    print(f"Общее количество инструментов: {total_instruments}")
    
    if unique_base_codes != total_instruments:
        print("⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!")
        duplicates = filtered_securities.groupby('base_code').size()
        duplicates = duplicates[duplicates > 1]
        print(f"Дублирующиеся base_code ({len(duplicates)}):")
        for base_code, count in duplicates.head(10).items():
            print(f"  {base_code}: {count} инструментов")
            # Показываем примеры дубликатов
            examples = filtered_securities[filtered_securities['base_code'] == base_code][['SECID', 'SHORTNAME', 'base_code']]
            print(f"    Примеры: {examples['SECID'].tolist()}")
    else:
        print("✅ Все base_code уникальны")
        
else:
    print('После фильтрации инструментов не найдено.')


Инструментов в whitelist: 44
Инструментов после фильтра по суффиксу Z5: 131
Инструментов после применения whitelist: 45
Примеры отфильтрованных инструментов:


,SECID,SHORTNAME,base_code,full_price
0,AFZ5,AFLT-12.25,AFLT,5793.0
1,AKZ5,AFKS-12.25,AFKS,14687.0
2,ALZ5,ALRS-12.25,ALRS,4627.0
3,BNZ5,BANE-12.25,BANE,1600.0
4,CMZ5,CBOM-12.25,CBOM,7539.0



Проверка уникальности по base_code:
Уникальных base_code: 44
Общее количество инструментов: 45
⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!
Дублирующиеся base_code (1):
  T: 2 инструментов
    Примеры: ['TBZ5', 'TIZ5']


## 5.1. Диалог для удаления дублирующихся инструментов

Если найдены дубликаты по base_code, создаем интерактивный диалог для выбора инструментов к удалению.

In [ ]:
def remove_duplicates_interactive(df, base_code_column='base_code'):
    """
    Интерактивная функция для удаления дублирующихся инструментов по base_code.
    
    Args:
        df: DataFrame с инструментами
        base_code_column: название колонки для проверки дубликатов
    
    Returns:
        DataFrame без дубликатов
    """
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import pandas as pd
    
    # Находим дубликаты
    duplicates = df.groupby(base_code_column).size()
    duplicates = duplicates[duplicates > 1]
    
    if len(duplicates) == 0:
        print("✅ Дубликатов не найдено!")
        return df
    
    print(f"🔍 Найдено {len(duplicates)} групп дубликатов:")
    
    # Создаем виджеты для каждой группы дубликатов
    widgets_list = []
    selected_to_remove = []
    
    for base_code in duplicates.index:
        group_data = df[df[base_code_column] == base_code]
        print(f"\n📋 Группа '{base_code}' ({len(group_data)} инструментов):")
        
        # Создаем чекбоксы для каждого инструмента в группе
        checkboxes = []
        for idx, row in group_data.iterrows():
            checkbox = widgets.Checkbox(
                value=False,
                description=f"{row['SECID']} - {row['SHORTNAME']} (цена: {row['full_price']})",
                style={'description_width': 'initial'}
            )
            checkboxes.append(checkbox)
            widgets_list.append(checkbox)
        
        # Показываем инструменты в группе
        display(widgets.VBox(checkboxes))
        
        # Сохраняем информацию о группе
        selected_to_remove.append({
            'base_code': base_code,
            'checkboxes': checkboxes,
            'group_data': group_data
        })
    
    # Кнопка для применения изменений
    apply_button = widgets.Button(
        description="🗑️ Удалить выбранные инструменты",
        button_style='danger',
        layout=widgets.Layout(width='300px')
    )
    
    def on_apply_clicked(b):
        clear_output(wait=True)
        print("🔄 Обрабатываем выбор...")
        
        # Собираем выбранные для удаления инструменты
        indices_to_remove = []
        
        for group_info in selected_to_remove:
            base_code = group_info['base_code']
            checkboxes = group_info['checkboxes']
            group_data = group_info['group_data']
            
            for i, checkbox in enumerate(checkboxes):
                if checkbox.value:  # Если чекбокс отмечен
                    # Находим индекс в исходном DataFrame
                    secid = group_data.iloc[i]['SECID']
                    idx = df[df['SECID'] == secid].index[0]
                    indices_to_remove.append(idx)
        
        if indices_to_remove:
            print(f"🗑️ Удаляем {len(indices_to_remove)} выбранных инструментов...")
            
            # Удаляем выбранные инструменты
            df_cleaned = df.drop(indices_to_remove).reset_index(drop=True)
            
            print(f"✅ Очистка завершена!")
            print(f"   Исходное количество: {len(df)}")
            print(f"   После очистки: {len(df_cleaned)}")
            print(f"   Удалено: {len(df) - len(df_cleaned)}")
            
            # Проверяем результат
            remaining_duplicates = df_cleaned.groupby(base_code_column).size()
            remaining_duplicates = remaining_duplicates[remaining_duplicates > 1]
            
            if len(remaining_duplicates) == 0:
                print("✅ Все дубликаты успешно удалены!")
            else:
                print(f"⚠️ Остались дубликаты: {len(remaining_duplicates)} групп")
            
            return df_cleaned
        else:
            print("ℹ️ Ничего не выбрано для удаления.")
            return df
    
    apply_button.on_click(on_apply_clicked)
    display(apply_button)
    
    return df  # Возвращаем исходный DataFrame, если ничего не выбрано

# Применяем функцию к отфильтрованным инструментам
if 'filtered_securities' in locals() and not filtered_securities.empty:
    print("🔧 Запускаем диалог удаления дубликатов...")
    filtered_securities_cleaned = remove_duplicates_interactive(filtered_securities)
    
    # Обновляем переменную
    filtered_securities = filtered_securities_cleaned
    print(f"\n📊 Итоговое количество инструментов: {len(filtered_securities)}")
else:
    print("❌ Нет данных для обработки дубликатов")


## 5.2. Альтернативный диалог (без ipywidgets)

Если ipywidgets недоступны, используем простой текстовый интерфейс для выбора дубликатов.

In [6]:
def remove_duplicates_simple(df, base_code_column='base_code'):
    """
    Простая функция для удаления дублирующихся инструментов по base_code.
    Использует текстовый интерфейс вместо ipywidgets.
    
    Args:
        df: DataFrame с инструментами
        base_code_column: название колонки для проверки дубликатов
    
    Returns:
        DataFrame без дубликатов
    """
    import pandas as pd
    
    # Находим дубликаты
    duplicates = df.groupby(base_code_column).size()
    duplicates = duplicates[duplicates > 1]
    
    if len(duplicates) == 0:
        print("✅ Дубликатов не найдено!")
        return df
    
    print(f"🔍 Найдено {len(duplicates)} групп дубликатов:")
    
    indices_to_remove = []
    
    for base_code in duplicates.index:
        group_data = df[df[base_code_column] == base_code]
        print(f"\n📋 Группа '{base_code}' ({len(group_data)} инструментов):")
        
        # Показываем инструменты в группе
        for i, (idx, row) in enumerate(group_data.iterrows()):
            print(f"  {i+1}. {row['SECID']} - {row['SHORTNAME']} (цена: {row['full_price']})")
        
        # Запрашиваем выбор
        print(f"\n❓ Какие инструменты из группы '{base_code}' удалить?")
        print("   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:")
        
        try:
            choice = input("Ваш выбор: ").strip()
            
            if choice.lower() == 'all':
                # Удаляем все кроме первого
                indices_to_remove.extend(group_data.index[1:].tolist())
                print(f"   ✅ Будет удалено {len(group_data)-1} инструментов (останется первый)")
            elif choice:
                # Парсим номера
                numbers = [int(x.strip()) for x in choice.split(',') if x.strip().isdigit()]
                if numbers:
                    # Проверяем валидность номеров
                    valid_numbers = [n for n in numbers if 1 <= n <= len(group_data)]
                    if valid_numbers:
                        # Получаем индексы для удаления
                        indices_to_remove.extend([group_data.index[n-1] for n in valid_numbers])
                        print(f"   ✅ Будет удалено {len(valid_numbers)} инструментов")
                    else:
                        print("   ⚠️ Некорректные номера, пропускаем группу")
                else:
                    print("   ⚠️ Некорректный ввод, пропускаем группу")
            else:
                print("   ⏭️ Группа пропущена")
                
        except (ValueError, KeyboardInterrupt):
            print("   ⚠️ Некорректный ввод или прерывание, пропускаем группу")
            continue
    
    if indices_to_remove:
        print(f"\n🗑️ Удаляем {len(indices_to_remove)} выбранных инструментов...")
        
        # Удаляем выбранные инструменты
        df_cleaned = df.drop(indices_to_remove).reset_index(drop=True)
        
        print(f"✅ Очистка завершена!")
        print(f"   Исходное количество: {len(df)}")
        print(f"   После очистки: {len(df_cleaned)}")
        print(f"   Удалено: {len(df) - len(df_cleaned)}")
        
        # Проверяем результат
        remaining_duplicates = df_cleaned.groupby(base_code_column).size()
        remaining_duplicates = remaining_duplicates[remaining_duplicates > 1]
        
        if len(remaining_duplicates) == 0:
            print("✅ Все дубликаты успешно удалены!")
        else:
            print(f"⚠️ Остались дубликаты: {len(remaining_duplicates)} групп")
        
        return df_cleaned
    else:
        print("ℹ️ Ничего не выбрано для удаления.")
        return df

# Попробуем использовать простую версию, если интерактивная не работает
try:
    if 'filtered_securities' in locals() and not filtered_securities.empty:
        print("🔧 Запускаем простой диалог удаления дубликатов...")
        filtered_securities_cleaned = remove_duplicates_simple(filtered_securities)
        
        # Обновляем переменную
        filtered_securities = filtered_securities_cleaned
        print(f"\n📊 Итоговое количество инструментов: {len(filtered_securities)}")
    else:
        print("❌ Нет данных для обработки дубликатов")
except Exception as e:
    print(f"❌ Ошибка при обработке дубликатов: {e}")
    print("Продолжаем с исходными данными...")


🔧 Запускаем простой диалог удаления дубликатов...
🔍 Найдено 1 групп дубликатов:

📋 Группа 'T' (2 инструментов):
  1. TBZ5 - T-12.25 (цена: 3222.0)
  2. TIZ5 - T-12.25 (цена: 3222.0)

❓ Какие инструменты из группы 'T' удалить?
   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:
   ✅ Будет удалено 1 инструментов

🗑️ Удаляем 1 выбранных инструментов...
✅ Очистка завершена!
   Исходное количество: 45
   После очистки: 44
   Удалено: 1
✅ Все дубликаты успешно удалены!

📊 Итоговое количество инструментов: 44


In [7]:
print(filtered_securities.shape)
filtered_securities

(44, 13)


,SECID,MINSTEP,STEPPRICE,PREVSETTLEPRICE,INITIALMARGIN,BUYSELLFEE,SCALPERFEE,SHORTNAME,full_price,CODE,PRICE,SELLDEPO,base_code
0,AFZ5,1.0,1.0,5793.0,1523.15,1.15,0.58,AFLT-12.25,5793.0,AFZ5,5793.0,1523.15,AFLT
1,AKZ5,1.0,1.0,14687.0,3857.79,2.91,1.46,AFKS-12.25,14687.0,AKZ5,14687.0,3857.79,AFKS
2,ALZ5,1.0,1.0,4627.0,1214.01,0.92,0.46,ALRS-12.25,4627.0,ALZ5,4627.0,1214.01,ALRS
3,BNZ5,1.0,1.0,1600.0,544.83,0.31,0.16,BANE-12.25,1600.0,BNZ5,1600.0,544.83,BANE
4,CMZ5,1.0,1.0,7539.0,2576.72,1.49,0.75,CBOM-12.25,7539.0,CMZ5,7539.0,2576.72,CBOM
5,FLZ5,1.0,1.0,8367.0,1541.90,1.65,0.83,FLOT-12.25,8367.0,FLZ5,8367.0,1541.90,FLOT
6,FSZ5,1.0,1.0,6850.0,1463.69,1.36,0.68,FEES-12.25,6850.0,FSZ5,6850.0,1463.69,FEES
7,GKZ5,1.0,1.0,1277.0,235.32,0.26,0.13,GMKN-12.25,1277.0,GKZ5,1277.0,235.32,GMKN
8,GZZ5,1.0,1.0,12441.0,2287.73,2.47,1.24,GAZR-12.25,12441.0,GZZ5,12441.0,2287.73,GAZR
9,HDZ5,1.0,1.0,3543.0,1209.21,0.70,0.35,HEAD-12.25,3543.0,HDZ5,3543.0,1209.21,HEAD


## 6. Создание основы для построения портфеля

Создаем основу для портфеля с расчетом весовых коэффициентов относительно индикаторного инструмента.


In [8]:
if SEC_IND_PRICE is not None and not filtered_securities.empty:
    print(f"Создаем основу портфеля с {NUM_PARAMS} параметрами для каждого инструмента")
    print(f"Индикаторный инструмент: {SEC_IND} (цена: {SEC_IND_PRICE:,.2f} руб.)")
    
    # Создаем список для хранения данных портфеля
    portfolio_basis = []
    
    for idx, row in filtered_securities.iterrows():
        sec_0 = row['SECID']
        sec_0_price = row['full_price']
        sec_0_GO = row['INITIALMARGIN']
        sec_1 = SEC_IND
        
        # Расчет весового коэффициента W_0
        W_0 = (sec_0_price / SEC_IND_PRICE * 1000000) // 1000 / 1000
        
        # Расчет float значений
        float_1 = max(1, int(sec_0_price / 100 * float_1_x))
        float_2 = max(1, int(sec_0_price / 100 * float_2_x))
        float_3 = float_3_x
        float_4 = float_4_x
        
        # Формирование колонки SM
        SM = f"{float_1};{float_2};{float_3};{float_4}"
        
        portfolio_basis.append({
            'Sec_0': sec_0,
            'sec_0_price': sec_0_price,
            'sec_0_GO': sec_0_GO,
            'sec_1': sec_1,
            'W_0': W_0,
            'SM': SM
        })
    
    # Преобразуем в DataFrame
    portfolio_basis_df = pd.DataFrame(portfolio_basis)
    
    print(f"\nСоздана основа портфеля для {len(portfolio_basis_df)} инструментов")
    print("\nПримеры записей:")
    # display(portfolio_basis_df.head())
    
    # Статистика по весовым коэффициентам
    print(f"\nСтатистика весовых коэффициентов W_0:")
    print(f"Минимум: {portfolio_basis_df['W_0'].min():.6f}")
    print(f"Максимум: {portfolio_basis_df['W_0'].max():.6f}")
    print(f"Среднее: {portfolio_basis_df['W_0'].mean():.6f}")
    print(f"Медиана: {portfolio_basis_df['W_0'].median():.6f}")
    
else:
    print("Ошибка: Не удалось создать основу портфеля.")
    if SEC_IND_PRICE is None:
        print("- Индикаторный инструмент не найден")
    if filtered_securities.empty:
        print("- Нет отфильтрованных инструментов")


Создаем основу портфеля с 3 параметрами для каждого инструмента
Индикаторный инструмент: MXZ5 (цена: 281,500.00 руб.)

Создана основа портфеля для 44 инструментов

Примеры записей:

Статистика весовых коэффициентов W_0:
Минимум: 0.002000
Максимум: 0.402000
Среднее: 0.054682
Медиана: 0.023000


## 7. Размножение строк по NUM_PARAMS

Создаем strategy_id для каждой комбинации инструмента и параметра, размножая строки по количеству параметров.


In [9]:
if 'portfolio_basis_df' in locals() and not portfolio_basis_df.empty:
    print("Размножаем строки по NUM_PARAMS...")
    
    # Создаем список параметров от 1 до NUM_PARAMS
    params_list = list(range(1, NUM_PARAMS + 1))
    print(f"Список параметров: {params_list}")
    
    # Создаем размноженные строки
    expanded_portfolio = []
    
    for idx, row in portfolio_basis_df.iterrows():
        for param_num in params_list:
            # Создаем strategy_id
            strategy_id = f"{row['Sec_0']}_{param_num}"
            
            # Копируем все данные строки
            expanded_row = row.copy()
            expanded_row['strategy_id'] = strategy_id
            expanded_row['param_num'] = param_num
            
            # Переупорядочиваем колонки, чтобы strategy_id была первой
            expanded_row = expanded_row.reindex(['strategy_id'] + [col for col in expanded_row.index if col != 'strategy_id'])
            
            expanded_portfolio.append(expanded_row)
    
    # Преобразуем в DataFrame
    expanded_portfolio_df = pd.DataFrame(expanded_portfolio)
    
    print(f"\nРазмножение завершено:")
    print(f"Исходных инструментов: {len(portfolio_basis_df)}")
    print(f"Параметров на инструмент: {NUM_PARAMS}")
    print(f"Итого строк: {len(expanded_portfolio_df)}")
    print(f"Ожидалось: {len(portfolio_basis_df) * NUM_PARAMS}")
    
    print("\nПримеры strategy_id:")
    sample_strategies = expanded_portfolio_df['strategy_id'].head(10).tolist()
    print(sample_strategies)
    
    print("\nПримеры записей:")
    display(expanded_portfolio_df[['strategy_id', 'Sec_0', 'param_num', 'W_0', 'SM']].head())
    
    # Проверяем уникальность strategy_id
    unique_strategies = expanded_portfolio_df['strategy_id'].nunique()
    total_strategies = len(expanded_portfolio_df)
    print(f"\nПроверка уникальности:")
    print(f"Уникальных strategy_id: {unique_strategies}")
    print(f"Общее количество строк: {total_strategies}")
    print(f"Совпадают: {unique_strategies == total_strategies}")
    
else:
    print("Ошибка: Не удалось размножить строки.")
    print("Убедитесь, что основа портфеля была создана успешно.")


Размножаем строки по NUM_PARAMS...
Список параметров: [1, 2, 3]

Размножение завершено:
Исходных инструментов: 44
Параметров на инструмент: 3
Итого строк: 132
Ожидалось: 132

Примеры strategy_id:
['AFZ5_1', 'AFZ5_2', 'AFZ5_3', 'AKZ5_1', 'AKZ5_2', 'AKZ5_3', 'ALZ5_1', 'ALZ5_2', 'ALZ5_3', 'BNZ5_1']

Примеры записей:


,strategy_id,Sec_0,param_num,W_0,SM
0,AFZ5_1,AFZ5,1,0.020,1;1;0.01;0.05
0,AFZ5_2,AFZ5,2,0.020,1;1;0.01;0.05
0,AFZ5_3,AFZ5,3,0.020,1;1;0.01;0.05
1,AKZ5_1,AKZ5,1,0.052,4;1;0.01;0.05
1,AKZ5_2,AKZ5,2,0.052,4;1;0.01;0.05



Проверка уникальности:
Уникальных strategy_id: 132
Общее количество строк: 132
Совпадают: True


In [10]:
expanded_portfolio_df.head()

,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,param_num
0,AFZ5_1,AFZ5,5793.0,1523.15,MXZ5,0.020,1;1;0.01;0.05,1
0,AFZ5_2,AFZ5,5793.0,1523.15,MXZ5,0.020,1;1;0.01;0.05,2
0,AFZ5_3,AFZ5,5793.0,1523.15,MXZ5,0.020,1;1;0.01;0.05,3
1,AKZ5_1,AKZ5,14687.0,3857.79,MXZ5,0.052,4;1;0.01;0.05,1
1,AKZ5_2,AKZ5,14687.0,3857.79,MXZ5,0.052,4;1;0.01;0.05,2


## 8. Формирование финального каркаса портфеля

Создаем финальную структуру портфеля с анализом результатов.


## 8.1. Тестирование новой функции build_portfolio_advanced

Теперь протестируем новую функцию `build_portfolio_advanced`, которая может работать с любым DataFrame и правильно распределять капитал по группам инструментов и параметрам.


In [11]:
final_portfolio = expanded_portfolio_df.copy()
# Импортируем новую функцию
from tvr_service.pipeline import build_portfolio_advanced

if 'final_portfolio' in locals() and not final_portfolio.empty:
    print("Тестируем новую функцию build_portfolio_advanced...")
    print(f"Исходные данные: {len(final_portfolio)} строк, {final_portfolio['Sec_0'].nunique()} уникальных инструментов")
    
    # Применяем новую функцию для распределения капитала
    portfolio_with_allocation = build_portfolio_advanced(
        data=final_portfolio,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_price",
        param_num_column="param_num"
    )
    
    print(f"Результат: {len(portfolio_with_allocation)} строк")
    print("\nПример результата:")
    display(portfolio_with_allocation[['strategy_id', 'Sec_0', 'param_num', 'sec_0_price', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].head(10))
    
elif 'expanded_portfolio_df' in locals() and not expanded_portfolio_df.empty:
    print("⚠️  final_portfolio не найден, но expanded_portfolio_df доступен.")
    print("Создаем final_portfolio из expanded_portfolio_df...")
    
    # Создаем final_portfolio из expanded_portfolio_df
    final_portfolio = expanded_portfolio_df.copy()
    print(f"Создан final_portfolio: {len(final_portfolio)} строк")
    
    # Теперь применяем новую функцию
    portfolio_with_allocation = build_portfolio_advanced(
        data=final_portfolio,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_price",
        param_num_column="param_num"
    )
    
    print(f"Результат: {len(portfolio_with_allocation)} строк")
    print("\nПример результата:")
    display(portfolio_with_allocation[['strategy_id', 'Sec_0', 'param_num', 'sec_0_price', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].head(10))
    
else:
    print("❌ Ошибка: Не найдены необходимые данные.")
    print("Доступные переменные:")
    available_vars = [var for var in locals() if 'portfolio' in var.lower() or 'expanded' in var.lower()]
    if available_vars:
        print(f"  - {', '.join(available_vars)}")
    else:
        print("  - Нет переменных с 'portfolio' или 'expanded' в названии")
    print("\nУбедитесь, что выполнили все предыдущие шаги блокнота.")


Тестируем новую функцию build_portfolio_advanced...
Исходные данные: 132 строк, 44 уникальных инструментов
Результат: 132 строк

Пример результата:


,strategy_id,Sec_0,param_num,sec_0_price,allocation,estimated_lots,used_capital,unused_capital
0,AFZ5_1,AFZ5,1,5793.0,22727.272727,3,17379.0,5348.272727
0,AFZ5_2,AFZ5,2,5793.0,22727.272727,3,17379.0,5348.272727
0,AFZ5_3,AFZ5,3,5793.0,22727.272727,3,17379.0,5348.272727
1,AKZ5_1,AKZ5,1,14687.0,22727.272727,1,14687.0,8040.272727
1,AKZ5_2,AKZ5,2,14687.0,22727.272727,1,14687.0,8040.272727
1,AKZ5_3,AKZ5,3,14687.0,22727.272727,1,14687.0,8040.272727
2,ALZ5_1,ALZ5,1,4627.0,22727.272727,4,18508.0,4219.272727
2,ALZ5_2,ALZ5,2,4627.0,22727.272727,4,18508.0,4219.272727
2,ALZ5_3,ALZ5,3,4627.0,22727.272727,4,18508.0,4219.272727
3,BNZ5_1,BNZ5,1,1600.0,22727.272727,14,22400.0,327.272727


In [12]:
# Анализ результатов распределения капитала
if 'portfolio_with_allocation' in locals():
    print("=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ КАПИТАЛА ===\n")
    
    # Общая статистика
    total_used = portfolio_with_allocation['used_capital'].sum()
    total_unused = portfolio_with_allocation['unused_capital'].sum()
    utilization_rate = total_used / CAPITAL * 100
    
    print(f"Общий капитал: {CAPITAL:,.0f} руб.")
    print(f"Использовано: {total_used:,.0f} руб. ({utilization_rate:.1f}%)")
    print(f"Неиспользовано: {total_unused:,.0f} руб. ({100-utilization_rate:.1f}%)")
    
    # Статистика по инструментам
    print(f"\n=== СТАТИСТИКА ПО ИНСТРУМЕНТАМ ===")
    instrument_stats = portfolio_with_allocation.groupby('Sec_0').agg({
        'allocation': 'first',  # allocation одинаковая для всех строк инструмента
        'estimated_lots': 'sum',
        'used_capital': 'sum',
        'unused_capital': 'sum',
        'param_num': 'count'
    }).rename(columns={'param_num': 'param_count'})
    
    print(f"Уникальных инструментов: {len(instrument_stats)}")
    print(f"Капитал на инструмент: {CAPITAL / len(instrument_stats):,.0f} руб.")
    print("\nТоп-5 инструментов по используемому капиталу:")
    top_instruments = instrument_stats.nlargest(5, 'used_capital')[['used_capital', 'estimated_lots', 'param_count']]
    display(top_instruments)
    
    # Пример распределения внутри одного инструмента
    print(f"\n=== ПРИМЕР РАСПРЕДЕЛЕНИЯ ВНУТРИ ИНСТРУМЕНТА ===")
    sample_instrument = portfolio_with_allocation['Sec_0'].iloc[0]
    sample_group = portfolio_with_allocation[portfolio_with_allocation['Sec_0'] == sample_instrument]
    print(f"Инструмент: {sample_instrument}")
    print(f"Цена: {sample_group['sec_0_price'].iloc[0]:,.0f} руб.")
    print(f"Параметров: {len(sample_group)}")
    
    sample_display = sample_group[['strategy_id', 'param_num', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].sort_values('param_num')
    display(sample_display)
    
    print(f"\nПроверка суммы лотов: {sample_group['estimated_lots'].sum()} лотов")
    expected_lots = int((CAPITAL / len(instrument_stats)) // sample_group['sec_0_price'].iloc[0])
    print(f"Ожидаемая сумма лотов: {expected_lots} лотов")
    print(f"Совпадает: {'✅' if sample_group['estimated_lots'].sum() == expected_lots else '❌'}")
    
else:
    print("Переменная portfolio_with_allocation не найдена.")


=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ КАПИТАЛА ===

Общий капитал: 3,000,000 руб.
Использовано: 2,069,205 руб. (69.0%)
Неиспользовано: 930,795 руб. (31.0%)

=== СТАТИСТИКА ПО ИНСТРУМЕНТАМ ===
Уникальных инструментов: 44
Капитал на инструмент: 68,182 руб.

Топ-5 инструментов по используемому капиталу:


,used_capital,estimated_lots,param_count
Sec_0,,,
CMZ5,67851.0,9,3
MVZ5,67743.0,117,3
PSZ5,67716.0,54,3
TBZ5,67662.0,21,3
X5Z5,67560.0,24,3



=== ПРИМЕР РАСПРЕДЕЛЕНИЯ ВНУТРИ ИНСТРУМЕНТА ===
Инструмент: AFZ5
Цена: 5,793 руб.
Параметров: 3


,strategy_id,param_num,allocation,estimated_lots,used_capital,unused_capital
0,AFZ5_1,1,22727.272727,3,17379.0,5348.272727
0,AFZ5_2,2,22727.272727,3,17379.0,5348.272727
0,AFZ5_3,3,22727.272727,3,17379.0,5348.272727



Проверка суммы лотов: 9 лотов
Ожидаемая сумма лотов: 11 лотов
Совпадает: ❌


In [ ]:
# Демонстрация гибкости функции: тестируем с другим столбцом для расчетов
if 'portfolio_with_allocation' in locals():
    print("=== ТЕСТИРОВАНИЕ С ДРУГИМ СТОЛБЦОМ ДЛЯ РАСЧЕТОВ ===\n")
    print("Сравниваем результаты при использовании 'sec_0_price' и 'sec_0_GO' для расчетов:\n")
    
    # Тестируем с sec_0_GO (гарантийное обеспечение)
    portfolio_with_go = build_portfolio_advanced(
        data=final_portfolio,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_GO",  # Используем гарантийное обеспечение
        param_num_column="param_num"
    )
    
    # Сравнение для одного инструмента
    sample_sec = final_portfolio['Sec_0'].iloc[0]
    
    print(f"Пример для инструмента: {sample_sec}")
    
    # Данные по цене
    price_group = portfolio_with_allocation[portfolio_with_allocation['Sec_0'] == sample_sec]
    price_summary = {
        'Основа расчета': f"sec_0_price ({price_group['sec_0_price'].iloc[0]:,.0f} руб.)",
        'Общие лоты': price_group['estimated_lots'].sum(),
        'Использованный капитал': price_group['used_capital'].sum(),
        'Неиспользованный капитал': price_group['unused_capital'].sum()
    }
    
    # Данные по ГО
    go_group = portfolio_with_go[portfolio_with_go['Sec_0'] == sample_sec]
    go_summary = {
        'Основа расчета': f"sec_0_GO ({go_group['sec_0_GO'].iloc[0]:,.0f} руб.)",
        'Общие лоты': go_group['estimated_lots'].sum(),
        'Использованный капитал': go_group['used_capital'].sum(),
        'Неиспользованный капитал': go_group['unused_capital'].sum()
    }
    
    comparison_df = pd.DataFrame([price_summary, go_summary], index=['По цене', 'По ГО'])
    display(comparison_df)
    
    print(f"\nВывод: Функция работает с любым столбцом для расчетов!")
    print(f"При использовании ГО получаем больше лотов, так как ГО обычно меньше полной цены контракта.")
    
else:
    print("Ошибка: portfolio_with_allocation не создан.")
